In [2]:
import os
import re
import torch
import glob
import numpy as np
import pandas as pd
import torch.optim as optim
import matplotlib.pyplot as plt
from RadarDataset import RadarDataset
from TripletConvolution import TCN, trainTCN

In [47]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

signalTypes = ['Barker', 'Costas', 'Frank', 'LFM', 'P1', 'P2', 'P3', 'P4', 'T1', 'T2', 'T3', 'T4']
TransformedTypes = {'DWT' : "/data/kiwan/Unknown_radar_detection/Adaptive_wavelet_transform/dataset-SPWVD-denoised-Adaptive_DWT",
                    'CWD' : "/data/kiwan/Unknown_radar_detection/Adaptive_wavelet_transform/240523_CWD-v1/",
                    'SAFI' : "/data/kiwan/Unknown_radar_detection/Adaptive_wavelet_transform/240523_SAFI-v1/",}
RawType = "/data/kiwan/dataset-CWD-50/"

signal_groups = {'v0': ['Barker', 'Costas', 'Frank', 'LFM', 'P1', 'P2', 'P3', 'P4'], 'v0i' : [0, 1, 2, 3, 4, 5, 6, 7],
                 'v1': ['Barker', 'Costas', 'Frank', 'LFM', 'T1', 'T2', 'T3', 'T4'], 'v1i' : [0, 1, 2, 3, 8, 9, 10, 11],
                 'v2': ['Barker', 'Costas', 'Frank', 'LFM', 'P1', 'P2', 'T3', 'T4'], 'v2i' : [0, 1, 2, 3, 4, 5, 10, 11],
                 'v3': ['Barker', 'Costas', 'Frank', 'LFM', 'P3', 'P4', 'T1', 'T2'], 'v3i' : [0, 1, 2, 3, 6, 7, 8, 9],}

In [3]:
transform = 'SAFI'
Dataset = RadarDataset(TransformedTypes[transform], signalTypes)

In [52]:
import re
import os
import glob

data_list = {}
fig_size = 5

for signal in signalTypes:
    signal_csvs = glob.glob(os.path.join(RawType, signal, '*.csv'))
    for signal_csv in signal_csvs:
        match = re.search(r'snr-?(\d+)', signal_csv)
        csv_snr = int(match.group(1)) if match else None
        
        data_iq = [complex(iq.replace('i', 'j')) for iq in np.array(pd.read_csv(signal_csv, header=None)).flatten()]
        data_real, data_img = zip(*[(iq.real, iq.imag) for iq in data_iq])
        data_list.setdefault(signal, {}).setdefault(csv_snr, []).append((data_real, data_img)) # {10: [('Barker', 10, '/data/kiwan/dataset-CWD-50/Barker/barker-snr-10-no00001.csv'), ...]}
        

"""
snr_max = 16
signalGroup = signal_groups['v0']
fig, axs = plt.subplots(len(signalGroup), len(range(0, snr_max+1, 2)), figsize=(25, len(signalGroup)//2 * 5))
for sI, signal in enumerate(signalGroup):
    for snI, snr in enumerate(range(0, snr_max+1, 2)):
        data_csv = pd.read_csv(data_list[signal][snr][0], header=None)
        
        data_iq = [complex(iq.replace('i', 'j')) for iq in np.array(data_csv).flatten()]
        data_real, data_img = zip(*[(iq.real, iq.imag) for iq in data_iq])
        
        axs[sI, snI].plot(data_real, label='real', color='blue', alpha=0.5)
        axs[sI, snI].plot(data_img, label='imag', color='red', alpha=0.5, linestyle='--')
        axs[sI, snI].set_xlabel('Sample')
        axs[sI, snI].set_ylabel('Amplitude')
        axs[sI, snI].set_title(f"{signal} (SNR: -{snr}dB)")
        axs[sI, snI].grid(True)
        axs[sI, snI].legend()

plt.tight_layout()
plt.show()
"""